# Garbage Classification

### Import Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.signal as signalpy
import tensorflow as tf
from PIL import Image
import keras_tuner as kt

## Data Preprocessing